### Market data refresh 

### Input Description

RAW OHLC data.

### Output  

Clean OHLC data in a hdf store

### Operations

This code takes a financial market data file and runs it through a processing pipeline. The following operations are carried out :

- Localise the time data to market time
- Merge with existing RAW data based on datetime
- Save the resulting RAW data to HDF5

In [4]:
#!pip install --upgrade "../../quantutils"
!pip install "../../marketinsights-price-aggregator"
import json, os, pandas
import quantutils.dataset.pipeline as ppl
from quantutils.api.datasource import MarketDataStore
import MIPriceAggregator.connectors as connectors
import pandas as pd
import numpy as np
import time
from datetime import datetime, date, timedelta

import warnings
from tables import NaturalNameWarning
warnings.filterwarnings('ignore', category=NaturalNameWarning)
from tqdm import tqdm

Processing /home/cwilkin/Development/repos/marketinsights-price-aggregator
  Preparing metadata (setup.py) ... done
  Created wheel for MIPriceAggregator: filename=MIPriceAggregator-1.0.0-py3-none-any.whl size=7471 sha256=263f6db3c6439a8b4d0594adf5717073dff9538b46069d91c1ab75521d659534
  Stored in directory: /home/cwilkin/.cache/pip/wheels/a3/66/dc/3a75635dd1cbb1bf931d0df38e458b1890be1666bbb5b88c7f
Successfully built MIPriceAggregator
  Attempting uninstall: MIPriceAggregator
    Found existing installation: MIPriceAggregator 1.0.0
    Uninstalling MIPriceAggregator-1.0.0:
      Successfully uninstalled MIPriceAggregator-1.0.0


In [5]:
def getConnector(connClass, connName, tz, options):
    connectorClass = getattr(connectors, connClass)
    connectorInstance = connectorClass(connName, tz, options)
    return connectorInstance

def fetchHistoricalData(mds, ds_file, start="1979-01-01", end="2050-01-01", records=200, delta=False):

    # Loop over datasources...
    # TODO: In chronological order

    datasources = json.load(open(ds_file))
    
    data = pd.DataFrame()
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["options"])

        for market in datasource["markets"]:

            for source in tqdm(market["sources"]):
                
                if delta:
                    try:
                        start = mds.aggregate([source["name"]], source["sample_unit"]).index[-1].strftime('%Y-%m-%d')
                    except:
                        print("Could not find " + source["name"])
                        start = "1979-01-01"
                
                    newData = dataConnector.getData(market, source, start)
                else:
                    newData = dataConnector.getData(market, source, start, end, records)
                
                if newData is not None:
                    
                    print("Adding " + source["name"] + " to " + market["name"] + " table")
                    
                    if mds is not None:  
                        mds.append(source["name"], newData, source["sample_unit"], update=True)
                    
                    data = ppl.merge(data, newData)
    return data


In [6]:
def fetchOptionData(datasources, start="1979-01-01", end="2050-01-01"):
            
    options = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'epic']))
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["opts"])

        for market in datasource["markets"]:

            for optionChain in tqdm(market["optionChains"]):
                
                optionData = dataConnector.getOptions(optionChain, appendUnderlying=False, start=start, end=end)
                
                if optionData is not None:                    
                    print("Adding " + optionChain["name"] + " to " + market["name"] + " table")    
                    
                    optionChain["options"] = json.loads(optionData.reset_index()[["epic","instrumentName","strike","type"]].to_json(orient="records"))
                    options = ppl.merge(options, optionData)                                    
        
        market["sources"].extend(json.loads(options.reset_index().assign(name=lambda x: x["epic"]).assign(sample_unit="D")[["name", "sample_unit"]].to_json(orient="records")))
    
    options = options.sort_values(ascending=[False, True, True], by=["Date_Time", "strike", "type"]).dropna()
    return options, datasources
    
def getOptionChains(datasource, root):
    
    dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["options"])
    
    chains = dataConnector.getOptionChains(root)
    
    # Update expiry dates
    dates = []
    for _,chain in chains.iterrows():
        if np.isnan(chain["expiry"]):
            info = dataConnector.getOptionInfo(chain["name"])
            dates.append(info["contractExpirationDate"])
        else:
            dates.append(str(date.today() + timedelta(int(chain["expiry"]) + 1)))
    chains["expiry"] = dates
    
    return chains

def appendOptionChainPrices(mds, ds_file):
    
    datasources = json.load(open(ds_file))
    
    options = pd.DataFrame(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Date_Time', u'epic']))
    data = pd.DataFrame()
    
    for datasource in datasources:

        dataConnector = getConnector(datasource["class"], datasource["name"], datasource["timezone"], datasource["options"])

        for market in datasource["markets"]:

            for optionChain in tqdm(market["optionChains"]):
                
                # Get todays prices from optionChain
                optionData = dataConnector.getOption(optionChain, appendUnderlying=False)
                                        
                if optionData is not None:
                    
                    print("Adding " + optionChain["name"] + " to " + market["name"] + " table")
                                        
                    df = optionData.replace(np.nan, 0).sort_values(["expiry","strike"]).reset_index()                    
                    for i,option in df.iterrows():
                        newData = df.iloc[i:i+1][["Date_Time","Open","High","Low","Close","Volume","OpenInterest"]].set_index("Date_Time")
                        if mds is not None:
                            mds.append(option["epic"], newData, "D", update=True)
                    
                    options = ppl.merge(options, optionData)
    return options


In [3]:
# Update options config file with all option chains
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))
market = datasources[0]["markets"][0]

chains = getOptionChains(datasources[0], market["optionRoot"])
market["options"] = json.loads(chains.to_json(orient="records"))

#with open(ds_file, 'w', encoding='utf-8') as f:
#    json.dump(datasources, f, ensure_ascii=False, indent=4)
    

NameError: name 'json' is not defined

In [67]:
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))

options, datasources = fetchOptionData(datasources)

with open(ds_file, 'w', encoding='utf-8') as f:
    json.dump(datasources, f, ensure_ascii=False, indent=4)

100%|██████████| 2/2 [00:09<00:00,  4.89s/it]

Adding CLX22 to Oil - US Crude table


In [70]:
ds_file = "../datasources/datasources_BarChartOption.json"
datasources = json.load(open(ds_file))

mkt = datasources[0]["markets"][0]
srcs = []
options = []
for source in mkt["sources"]:
    srcs.append({"name": source["name"], "sample_unit": source["sample_unit"]})
    strike = float(source["name"][5:-1])
    type = source["name"][-1].lower()
    if type == 'c':
        typeString = "Call"
    else:
        typeString = "Put"
    options.append({
        "epic": source["name"],
        "instrumentName": "Crude Oil WTI Oct '22 {} {}".format(strike, typeString),
        "strike": strike,
        "type": type
    })

mkt["sources"] = srcs
mkt["optionChains"][0]["options"] = options

with open(ds_file, 'w', encoding='utf-8') as f:
    json.dump(datasources, f, ensure_ascii=False, indent=4)

In [15]:
#fetchHistoricalData(None, "../datasources/datasources_MDS.json", end="2022-09-15")
ds_file = "../datasources/datasources_MDS.json"

options, datasources = fetchOptionData(json.load(open(ds_file)), end="2022-09-15")

  0%|          | 0/1 [00:00<?, ?it/s]

Loading data from CLV22 in ../datastore/data.hdf
Resampling to D periods
Resampling to D periods
Adding CLV2|210P to Option CLV22
Adding CLV2|220P to Option CLV22
Adding CLV2|230P to Option CLV22
Adding CLV2|240P to Option CLV22
Adding CLV2|250P to Option CLV22
Adding CLV2|260P to Option CLV22
Adding CLV2|270P to Option CLV22
Adding CLV2|275P to Option CLV22
Adding CLV2|280P to Option CLV22
Adding CLV2|290P to Option CLV22
Adding CLV2|300P to Option CLV22
Adding CLV2|310P to Option CLV22
Adding CLV2|315P to Option CLV22
Adding CLV2|320P to Option CLV22
Adding CLV2|325P to Option CLV22
Adding CLV2|330P to Option CLV22
Adding CLV2|340P to Option CLV22
Adding CLV2|345P to Option CLV22
Adding CLV2|350P to Option CLV22
Adding CLV2|355P to Option CLV22
Adding CLV2|360P to Option CLV22
Adding CLV2|365P to Option CLV22
Adding CLV2|370P to Option CLV22
Adding CLV2|375P to Option CLV22
Adding CLV2|380P to Option CLV22
Adding CLV2|390P to Option CLV22
Adding CLV2|395P to Option CLV22
Adding CLV2|

Adding CLV2|1050P to Option CLV22
Adding CLV2|1055P to Option CLV22
Adding CLV2|1055C to Option CLV22
Adding CLV2|1060C to Option CLV22
Adding CLV2|1060P to Option CLV22
Adding CLV2|1065C to Option CLV22
Adding CLV2|1065P to Option CLV22
Adding CLV2|1070C to Option CLV22
Adding CLV2|1070P to Option CLV22
Adding CLV2|1075C to Option CLV22
Adding CLV2|1075P to Option CLV22
Adding CLV2|1080C to Option CLV22
Adding CLV2|1080P to Option CLV22
Adding CLV2|1085P to Option CLV22
Adding CLV2|1085C to Option CLV22
Adding CLV2|1090P to Option CLV22
Adding CLV2|1090C to Option CLV22
Adding CLV2|1095C to Option CLV22
Adding CLV2|1095P to Option CLV22
Adding CLV2|1100C to Option CLV22
Adding CLV2|1100P to Option CLV22
Adding CLV2|1105C to Option CLV22
Adding CLV2|1105P to Option CLV22
Adding CLV2|1110C to Option CLV22
Adding CLV2|1110P to Option CLV22
Adding CLV2|1115C to Option CLV22
Adding CLV2|1115P to Option CLV22
Adding CLV2|1120C to Option CLV22
Adding CLV2|1120P to Option CLV22
Adding CLV2|11

100%|██████████| 1/1 [02:26<00:00, 146.49s/it]


In [35]:
options.to_pickle("tmp.pkl")

In [36]:
import plotly.express as px
df = px.data.gapminder()

In [39]:
np.array([1,2,3]) * 2

array([2, 4, 6])

In [13]:
# Local Options
mds = MarketDataStore(location="../datastore")
options = fetchHistoricalData(mds, "../datasources/datasources_BarChart.json", records=300)
options

  0%|          | 0/1 [00:00<?, ?it/s]

Converting from UTC to UTC
Adding CLV22 to Oil - US Crude table
Request to add data to table: CLV22
Table found: CLV22
Resampling to D periods
Re-writing table data for update...
Update complete


100%|██████████| 1/1 [00:05<00:00,  5.86s/it]


,Open,High,Low,Close,Volume,OpenInterest
Date_Time,,,,,,
2021-07-12 00:00:00+00:00,64.69,64.69,64.39,64.69,156,20236
2021-07-13 00:00:00+00:00,65.06,65.90,65.05,65.90,1660,21135
2021-07-14 00:00:00+00:00,64.66,64.66,64.33,64.66,782,21751
2021-07-15 00:00:00+00:00,64.30,64.30,64.03,64.03,770,21741
2021-07-16 00:00:00+00:00,64.49,64.49,63.81,64.49,422,21695
...,...,...,...,...,...,...
2022-09-12 00:00:00+00:00,86.25,89.10,85.16,87.78,275104,131037
2022-09-13 00:00:00+00:00,88.09,89.31,85.06,87.31,347998,107650
2022-09-14 00:00:00+00:00,87.94,90.19,86.18,88.48,335381,79882


In [7]:
# Local
mds = MarketDataStore(location="../datastore")
fetchHistoricalData(mds, "../datasources/datasources.json")
#refreshDatasources(mds, "../datasources/datasources_BarChart.json", start="2022-09-01", end="2022-09-020")

  0%|          | 0/1 [00:00<?, ?it/s]

Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Adding WallSt-hourly to DOW table
Request to add data to table: WallSt-hourly
Table found: WallSt-hourly
Resampling to H periods
Re-writing table data for update...
Update complete


  0%|          | 0/1 [00:00<?, ?it/s]

Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Converting from Europe/London to UTC
Adding SP500-hourly to SPY table
Request to add data to table: SP500-hourly
Table found: SP500-hourly
Resampling to H periods
Re-writing table data for update...
Update complete


  0%|          | 0/1 [00:00<?, ?it/s]

Converting from US/Eastern to UTC
Converting from US/Eastern to UTC
Converting from US/Eastern to UTC
Adding D&J-IND to DOW table
Request to add data to table: D&J-IND
Resampling to 5min periods
Creating new table for data...
Update complete


  0%|          | 0/1 [00:00<?, ?it/s]

Converting from US/Eastern to UTC
Converting from US/Eastern to UTC
Converting from US/Eastern to UTC
Adding SANDP-500 to SPY table
Request to add data to table: SANDP-500
Resampling to 5min periods
Creating new table for data...
Update complete


100%|██████████| 1/1 [00:55<00:00, 55.56s/it]


,Open,High,Low,Close
Date_Time,,,,
2013-01-01 05:00:00+00:00,1426.19,1426.19,1426.19,1426.19
2013-01-01 22:15:00+00:00,1426.19,1426.19,1426.19,1426.19
2013-01-02 05:00:00+00:00,1426.19,1426.19,1426.19,1426.19
2013-01-02 14:30:00+00:00,1426.19,1452.90,1426.19,1452.90
2013-01-02 14:35:00+00:00,1452.96,1455.35,1452.96,1454.79
...,...,...,...,...
2018-08-03 09:00:00+00:00,2826.70,2829.00,2826.20,2829.00
2018-08-03 10:00:00+00:00,2828.70,2830.50,2828.70,2829.70
2018-08-03 11:00:00+00:00,2830.00,2836.00,2829.70,2833.00


In [15]:
# Remote (cluster)
mds = MarketDataStore(remote=True, location="http://pricestore.192.168.1.203.nip.io")
#fetchHistoricalData(mds, "../datasources", "datasources.json")

In [ ]:
# Remote (localhost)
mds = MarketDataStore(remote=True, location="http://localhost:8080")
refreshMarketData(mds, "../datasources", "datasources.json")

In [11]:
df2 = mds.aggregate(["D&J-IND","SANDP-500"], "H", "1979-01-01", "2050-01-01", debug=True)
df2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Open,High,Low,Close
2013-01-02 15:00:00+00:00,13366.480000,13374.820000,13338.680000,13345.450000
2013-01-02 16:00:00+00:00,13345.600000,13353.750000,13321.640000,13321.770000
2013-01-02 17:00:00+00:00,13321.760000,13339.250000,13320.280000,13326.660000
2013-01-02 18:00:00+00:00,13326.640000,13336.060000,13322.510000,13329.190000
2013-01-02 19:00:00+00:00,13329.090000,13339.820000,13318.190000,13331.800000
...,...,...,...,...
2018-03-19 16:00:00+00:00,24632.111675,24650.292289,24554.939070,24583.990050
2018-03-19 17:00:00+00:00,24579.349894,24585.490101,24473.096306,24568.679533
2018-03-19 18:00:00+00:00,24570.309589,24590.980286,24455.305705,24533.498346
2018-03-19 19:00:00+00:00,24535.438411,24638.041875,24492.786971,24623.971400


In [8]:
df1 = mds.aggregate(["D&J-IND","SANDP-500"], "H", "1979-01-01", "2050-01-01", debug=True)
df1

Loading data from D&J-IND in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods
Loading data from SANDP-500 in ../datastore/data.hdf
Resampling to H periods
Resampling to H periods


,Open,High,Low,Close
Date_Time,,,,
2013-01-02 15:00:00+00:00,13366.480000,13374.820000,13338.680000,13345.450000
2013-01-02 16:00:00+00:00,13345.600000,13353.750000,13321.640000,13321.770000
2013-01-02 17:00:00+00:00,13321.760000,13339.250000,13320.280000,13326.660000
2013-01-02 18:00:00+00:00,13326.640000,13336.060000,13322.510000,13329.190000
2013-01-02 19:00:00+00:00,13329.090000,13339.820000,13318.190000,13331.800000
...,...,...,...,...
2018-03-19 16:00:00+00:00,24632.111675,24650.292289,24554.939070,24583.990050
2018-03-19 17:00:00+00:00,24579.349894,24585.490101,24473.096306,24568.679534
2018-03-19 18:00:00+00:00,24570.309589,24590.980286,24455.305705,24533.498346


In [17]:
mds.get("CLV2|900C")

,Open,High,Low,Close,Volume,OpenInterest
2021-11-30 00:00:00+00:00,3.53,2.82,2.55,3.53,0,200
2021-12-01 00:00:00+00:00,2.10,2.31,2.10,2.31,1,201
2021-12-02 00:00:00+00:00,2.01,2.01,2.01,2.01,0,201
2021-12-03 00:00:00+00:00,2.03,2.03,2.03,2.03,0,201
2021-12-04 00:00:00+00:00,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...
2022-09-11 00:00:00+00:00,NaN,NaN,NaN,NaN,0,0
2022-09-12 00:00:00+00:00,0.90,1.41,0.46,0.87,3923,5401
2022-09-13 00:00:00+00:00,0.95,1.31,0.22,0.48,3200,5824
2022-09-14 00:00:00+00:00,0.64,1.24,0.28,0.49,2183,6218


In [20]:
!pipdeptree



Warning!!! Possibly conflicting dependencies found:
* -ensorflow==1.10.0
 - setuptools [required: <=39.1.0, installed: 50.3.2]
 - numpy [required: >=1.13.3,<=1.14.5, installed: 1.19.5]
* arlo==1.2.50
 - urllib3 [required: ==1.24, installed: 1.25.11]
------------------------------------------------------------------------
-ensorflow==1.10.0
  - absl-py [required: >=0.1.6, installed: 0.9.0]
    - six [required: Any, installed: 1.16.0]
  - astor [required: >=0.6.0, installed: 0.6.2]
  - gast [required: >=0.2.0, installed: 0.3.3]
  - grpcio [required: >=1.8.6, installed: 1.31.0]
    - six [required: >=1.5.2, installed: 1.16.0]
  - numpy [required: >=1.13.3,<=1.14.5, installed: 1.19.5]
  - protobuf [required: >=3.6.0, installed: 3.13.0]
    - setuptools [required: Any, installed: 50.3.2]
    - six [required: >=1.9, installed: 1.16.0]
  - setuptools [required: <=39.1.0, installed: 50.3.2]
  - six [required: >=1.10.0, installed: 1.16.0]
  - tensorboard [required: >=1.10.0,<1.11.0, installed: 